In [1]:
import foldermerge
from pathlib import Path


root = Path().absolute().parent / "tests"

foldermerge.core.RESULTS_PATH = root / "results"

In [2]:
fm = foldermerge.FolderMerger(root / "integration" / "fixtures" / "mainfolder", root / "integration" / "fixtures" / "dupefolder1")

loading <Folder with name 626ed55d and path : c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\mainfolder>
loading <Folder with name fbd1a246 and path : c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1>
Finding all files in the repo c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\mainfolder


Searching: 2it [00:00, 500.42it/s]


saving <Folder with name 626ed55d and path : c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\mainfolder>
Finding all files in the repo c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1


Searching: 0it [00:00, ?it/s]

Searching: 2it [00:00, ?it/s]

saving <Folder with name fbd1a246 and path : c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1>
loading <foldermerge.core.FolderComparator object at 0x000001F520FC74D0>
saving <foldermerge.core.FolderComparator object at 0x000001F520FC74D0>


In [3]:
print(fm.report()[0])

Report of contents for repo fbd1a246 at c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1:
 - 9 total files found.
 - 4 identical files (will be deleted)
 - 1 inexistant files (will be copied in main)
 - 3 moved files (same content, different location) (tbd)
 - 1 changed files (same location, different content) (tbd)



In [4]:
comp = fm.folders.child(0).comparisons[fm.folders.main.name]

In [5]:
comp.get_changed_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
-2308957706120499499,content_changed_file_1.jpg,content_changed_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\content_changed_file_1.jpg,[images],1.706356e+09,1.707252e+09,1.707252e+09,00add8e211807e550a2a64807720543ac99813e9,[6266954397020811162],[]


In [6]:
comp.get_identical_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
-4075677595826027235,conflict_copy_file_1.jpg,conflict_copy_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\conflict_copy_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,0efc03bf785344023dd734b7119a91182178b8f0,[6214847964681687045],[6214847964681687045]
-441958247532708296,identical_file_1.jpg,identical_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,b3b29122e5dda4203d1e6fadc8ad34eb56e69845,[3541294403082671688],[3541294403082671688]
-1551860728720896224,identical_file_2.jpg,identical_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,c939af2ee954153e0290f57ae7761812754f7caf,[-8052198615219073956],[-8052198615219073956]
5092528099338450049,identical_file_3.jpg,identical_file_3,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_3.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,7dbd73a7133ccbd552f776da91da227ff5e121d0,[-6447277586661499169],[-6447277586661499169]


In [7]:
comp.get_inexistant_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
-8378644071822432228,inexistant_file_1.jpg,inexistant_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\inexistant_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,361151bd95808a23fdcff4d4adeb55e186998d48,[],[]


In [8]:
comp.get_moved_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
3568062500631960647,conflict_copy_duplicated_file_1.jpg,conflict_copy_duplicated_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\conflict_copy_duplicated_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,0efc03bf785344023dd734b7119a91182178b8f0,[],[6214847964681687045]
-1609281068694028568,renamed_file_1.jpg,renamed_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\renamed_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,58b24d71528352e2b4b46f497f3b0d87e5775877,[],[-275924656751009016]
7304253802138092403,renamed_file_2.jpg,renamed_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\renamed_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,5877ffde9d973af3a6c17254c66d54602cd634e9,[],[8995852696255613759]
